In [ ]:
import numpy as np
import pandas as pd
import pickle

# Carga de datos

In [ ]:
with open("C:/Users/Lenovo/Documents/GitHub/Datasets/Kaggle_Signals_ITBA/signals_train.pkl", 'rb') as f:
        lat=pickle.load(f)
with open("C:/Users/Lenovo/Documents/GitHub/Datasets/Kaggle_Signals_ITBA/annotations_train.pkl", 'rb') as f:
        ann=pickle.load(f)
with open("C:/Users/Lenovo/Documents/GitHub/Datasets/Kaggle_Signals_ITBA/patients_train.pkl", 'rb') as f:
        pat=pickle.load(f)

#print(type(lat))
#print(np.shape(lat))

# Aliviano el dataset

In [ ]:
# Selecciono los indices de 2000 latidos (500 de cada tipo)

print('R')
r = []
for index, value in enumerate(ann):
    if ann[index]=='R':
        #print(f'{index}: {value}')
        r.append(index)
R = r[:500]
print(R)
print(len(R))


print('N')
n = []
for index, value in enumerate(ann):
    if ann[index]=='N':
        n.append(index)
N = n[:500]
print(N)
print(len(N))
        

print('A')
a = []
for index, value in enumerate(ann):
    if ann[index]=='A':
        a.append(index)
A = a[:500]
print(A)
print(len(A))
        

print('V')
v = []
for index, value in enumerate(ann):
    if ann[index]=='V':
        v.append(index)
V = v[:500]
print(V)
print(len(V))

T = R + N + A + V
print(T)
print(len(T))
        


In [8]:
#print(type(lat))
#print(len(lat))
#print(type(lat[0]))
#print(len(lat[0]))
#print(type(lat[0][0]))
#print(len(lat[0][0]))
#print(type(lat[0][0][0]))
#print('>>>object of type numpy.float64 has no len()')
#print('\n')

In [ ]:
lat1 = np.array(lat)
latidos = lat1[T]
print(latidos.shape)

In [ ]:
ann1 = np.array(ann)
notas = ann1[T]
print(notas.shape)

In [ ]:
pat1 = np.array(pat)
paciente = pat1[T]
print(paciente.shape)

In [ ]:
print(type(latidos))
print(len(latidos))
print(latidos[0][100])

In [ ]:
l = len(latidos)
print(l)
latidos_final = []
for i in latidos:
    v = i[100:150]
    #print(np.shape(v))
    #print(type(v))
    latidos_final.append(v)

In [ ]:
print(np.shape(latidos_final))

In [ ]:
# Grafico el latido nro lat_idx
import numpy as np
from random import randint
import matplotlib.pyplot as plot
lat_idx=randint(0,len(latidos_final)) #randint(0,len(lat))
f, axarr = plot.subplots(12, sharex=True)
f.set_figheight(15)
f.set_figwidth(15)
print("Latido nro: {}".format(lat_idx))
print("El latido es del tipo: {}".format(notas[lat_idx])) #format(ann[lat_idx]))
print("El latido pertence al paciente {}".format(paciente[lat_idx])) #format(pat[lat_idx]))
for i in range(12):
    axarr[i].plot(latidos_final[lat_idx].T[i]) #axarr[i].plot(lat[lat_idx].T[i])
plot.show()

In [ ]:
import numpy as np
x = np.array(latidos_final)
print(x.shape)
y = np.array(notas)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.1)

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
from keras.callbacks import ModelCheckpoint 
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from helper import PlotLosses

model = Sequential()
model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(500, name='Salida1'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(4, activation='softmax', name='Salida5'))
model.summary()

In [ ]:
Adam = optimizers.adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.00001)
model.compile(loss = 'categorical_crossentropy', optimizer=Adam, metrics=['accuracy'])

In [ ]:
plot_losses = PlotLosses(plot_interval=1, evaluate_interval=20, x_val=x_valid, y_val_categorical=y_valid)
checkpointer = ModelCheckpoint(filepath='latidos.hdf5', verbose=1, save_best_only=True)

In [ ]:
print(x_valid.shape)
print(y_valid.shape)

In [ ]:
batch_size = 1024
epochs=5 #14
model.fit(x_train, 
                       y_train,
                       epochs=epochs , batch_size=batch_size, 
                       verbose=1, 
                       validation_data = (x_valid, y_valid),
                       callbacks=[plot_losses, checkpointer],
                      )